In [2]:
import ezc3d
cmj_1 = ezc3d.c3d("/Users/harrietdray/Baseline/Tash_CMJ1.c3d")
cmj_2 = ezc3d.c3d("/Users/harrietdray/Baseline/Tash_CMJ2.c3d")
cmj_3 = ezc3d.c3d("/Users/harrietdray/Baseline/Tash_CMJ3.c3d")

labels = cmj_1['parameters']['ROTATION']['LABELS']['value'] # List of position/rotation names
# Let's explore the structure of your C3D file


print("=== C3D File Structure ===")
print("Top level keys:", list(cmj_1.keys()))

print("\n=== Data Structure ===")
if 'data' in cmj_1:
    print("Data keys:", list(cmj_1['data'].keys()))

    # Check each data type
    for key in cmj_1['data'].keys():
        data = cmj_1['data'][key]
        print(f"{key} shape: {data.shape if hasattr(data, 'shape') else 'No shape'}")

print("\n=== Parameters Structure ===")
if 'parameters' in cmj_1:
    print("Parameter groups:", list(cmj_1['parameters'].keys()))
    
    # Look for rotation-related parameters
    for group in cmj_1['parameters'].keys():
        if 'ROTATION' in group.upper() or 'ANALOG' in group.upper():
            print(f"\n{group} parameters:")
            if 'LABELS' in cmj_1['parameters'][group]:
                labels_data = cmj_1['parameters'][group]['LABELS']
                print(f"  Labels: {labels_data.get('value', 'No value key')}")

print("\n=== Your Current Labels ===")
print("Labels source:", cmj_1['parameters']['ROTATION']['LABELS']['value'][:5], "...")
print("Total labels:", len(labels))

KeyError: 'ROTATION'

In [4]:
import ezc3d, re

c3d = ezc3d.c3d("/Users/harrietdray/Baseline/Tash_CMJ1.c3d")

# 1) What groups are present?
print("Groups:", list(c3d['parameters'].keys()))

# 2) Common label locations
point_labels  = c3d['parameters'].get('POINT', {}).get('LABELS', {}).get('value', [])
analog_labels = c3d['parameters'].get('ANALOG', {}).get('LABELS', {}).get('value', [])
print(f"POINT labels: {len(point_labels)}  ANALOG labels: {len(analog_labels)}")

# 3) Shapes
print("POINT data shape:", c3d['data']['points'].shape)   # (4, n_points, n_frames)
print("ANALOG data shape:", c3d['data']['analogs'].shape) # (n_subframes, n_analog, n_frames)

# 4) Find likely angle channels (case-insensitive)
angle_pat = re.compile(r'angle|rot', re.I)
angle_point_idx  = [i for i,l in enumerate(point_labels)  if angle_pat.search(l)]
angle_analog_idx = [i for i,l in enumerate(analog_labels) if angle_pat.search(l)]
print("Angle POINT idx+labels:",  [(i, point_labels[i])  for i in angle_point_idx][:20])
print("Angle ANALOG idx+labels:", [(i, analog_labels[i]) for i in angle_analog_idx][:20])

# 5) Access by index
points = c3d['data']['points']    # 4 x n_points x n_frames
analogs = c3d['data']['analogs']  # n_subframes x n_analog x n_frames

# Example: get first angle from POINTS (if any). Values are in mm for markers; for model outputs units may vary.
if angle_point_idx:
    i = angle_point_idx[0]
    angle_point_series = points[0, i, :]  # X row (models may store angles per axis as separate points)
    print("POINT angle sample:", angle_point_series[:10])

# Example: get first angle from ANALOGS (common for Nexus-exported joint angles in deg)
if angle_analog_idx:
    j = angle_analog_idx[0]
    angle_analog_series = analogs[:, j, :].reshape(-1)  # flatten subframes
    print("ANALOG angle sample:", angle_analog_series[:20])

Groups: ['TRIAL', 'SUBJECTS', 'POINT', 'ANALOG', 'FORCE_PLATFORM', 'EVENT_CONTEXT', 'EVENT', 'MANUFACTURER', 'ANALYSIS', 'PROCESSING']
POINT labels: 135  ANALOG labels: 56
POINT data shape: (4, 135, 814)
ANALOG data shape: (1, 56, 4070)
Angle POINT idx+labels: [(64, 'LHipAngles'), (65, 'LKneeAngles'), (66, 'LAnkleAngles'), (67, 'LForeFootAngles'), (68, 'RHipAngles'), (69, 'RKneeAngles'), (70, 'RAnkleAngles'), (71, 'RForeFootAngles'), (72, 'LShoulderAngles'), (73, 'LNeckAngles'), (74, 'RShoulderAngles'), (75, 'LFootProgressAngles'), (76, 'RFootProgressAngles'), (77, 'LPelvisAngles'), (78, 'RPelvisAngles')]
Angle ANALOG idx+labels: []
POINT angle sample: [5.78662014 5.78038931 5.80727005 5.80916977 5.83304977 5.79849005
 5.7392602  5.75094986 5.76609993 5.80551004]


In [3]:
# Let's debug the actual structure of your rotation data
print("=== DEBUGGING ROTATION DATA STRUCTURE ===")
rotation_data = cmj_1['data']['rotations']
print(f"Rotation data shape: {rotation_data.shape}")
print(f"Data type: {type(rotation_data)}")

# Let's see what the first few values look like
print(f"First 20 values of rotation_data[:, 0, 0]: {rotation_data[:20, 0, 0]}")

# Check if the data might be stored differently
print(f"Min value in rotation data: {rotation_data.min()}")
print(f"Max value in rotation data: {rotation_data.max()}")

# Let's look at the labels more carefully
print(f"\nLabels (first 10): {labels[:10]}")
print(f"Total labels: {len(labels)}")

# Let's try to understand the data better
print(f"Shape breakdown:")
print(f"  Dimension 0 (should be 16 for 4x4 matrix): {rotation_data.shape[0]}")
print(f"  Dimension 1 (frames): {rotation_data.shape[1]}")  
print(f"  Dimension 2 (joints): {rotation_data.shape[2]}")

# The issue might be that it's not 16 elements per matrix
# Let's check if each "joint" actually contains multiple matrices
single_joint_data = rotation_data[:, 0, 0]  # All data for first joint, first frame
print(f"\nSingle joint data size: {len(single_joint_data)}")
print(f"This might represent {len(single_joint_data)/16} matrices of 4x4 each")

=== DEBUGGING ROTATION DATA STRUCTURE ===
Rotation data shape: (4, 4, 0, 0)
Data type: <class 'numpy.ndarray'>


IndexError: index 0 is out of bounds for axis 2 with size 0